# Alignment optimisation

## 1 Setup

Flags

In [1]:
# None

Setup the database

In [2]:
import os, sys
sys.path.insert(1, os.path.abspath('../..'))
import analysis

DB_NAME = 'spreadr_' + os.path.split(os.path.abspath(os.path.curdir))[1]
GOLD_ALIGNMENTS = '../../data/alignments/{}/sebastien-lerique.csv'.format(DB_NAME)
analysis.setup(DB_NAME)
print('Database:', DB_NAME)

Database: spreadr_exp_3


In [3]:
#import random
import itertools
import csv
import json

import numpy as np
from scipy import optimize
from nltk.metrics import edit_distance
from frozendict import frozendict
#import matplotlib.pyplot as plt
#%matplotlib inline
#from mpl_toolkits.mplot3d import Axes3D
from progressbar import ProgressBar

from gists.models import Sentence
from analysis import settings, transformations

## 2 Optimise for hand-coded alignments

### 2.1 Framework setup

In [4]:
gold_sentences = []
gold_alignments = []
with open(GOLD_ALIGNMENTS, 'r') as gold_file:
    reader = csv.DictReader(gold_file)
    for row in reader:
        sentence = Sentence.objects.get(id=row['sentence_id'])
        gold_sentences.append(sentence)
        int_alignment = (json.loads(row['parent_coding']),
                         json.loads(row['sentence_coding']))
        alignment = transformations.int_decode_scoreless_alignment(
            sentence.parent.tokens, sentence.tokens, int_alignment)
        alignment += (0,)
        # One gold alignment per sentence
        gold_alignments.append([alignment])

In [5]:
def alignments(sentences, parameters):
    frozen_parameters = frozendict(parameters)
    return [transformations.align_lemmas(s.parent.tokens, s.tokens,
                                         parameters=frozen_parameters)
            for s in sentences]

In [6]:
def distance(alignment1, alignment2):
    seq1A, seq1B = alignment1[:2]
    seq2A, seq2B = alignment2[:2]
    seq1A = list(map(id, seq1A))
    seq1B = list(map(id, seq1B))
    seq2A = list(map(id, seq2A))
    seq2B = list(map(id, seq2B))
    return (edit_distance(seq1A, seq2A) + edit_distance(seq1B, seq2B)) / 2

In [7]:
BASE_COMPARE_FACTOR = 1
def x2parameters(x):
    return frozendict({
        'COMPARE_FACTOR': BASE_COMPARE_FACTOR,
        'COMPARE_ORIGIN': x[0] * BASE_COMPARE_FACTOR,
        'GAP_OPEN': (x[1] + x[2]) * BASE_COMPARE_FACTOR,
        'GAP_EXTEND': x[2] * BASE_COMPARE_FACTOR,
        'EXCHANGE': None,
    })

def parameters2x(parameters):
    return (np.array([parameters['COMPARE_ORIGIN'],
                      parameters['GAP_OPEN'] - parameters['GAP_EXTEND'],
                      parameters['GAP_EXTEND']])
            / parameters['COMPARE_FACTOR'])

In [8]:
def objective(x, sentences, ref_alignments):
    x_alignments = alignments(sentences, x2parameters(x))
    distances = []
    for ref_as, x_as in zip(ref_alignments, x_alignments):
        if len(x_as) == 0:
            # Add an empty alignment if there are none
            x_as = [([], [])]
        # Or use max+mean
        distances.append(np.max([distance(ref_a, x_a) for ref_a, x_a
                                 in itertools.product(ref_as,  x_as)]))
    return np.sum(distances)

### 2.2 Brute force

#### 2.2.1 On large boundaries, with discretization=10

In [9]:
x_bounds = [
    (-1, -.01), # COMPARE_ORIGIN / COMPARE_FACTOR
    (-1, -.01), # (GAP_OPEN - GAP_EXTEND) / COMPARE_FACTOR
    (-1, -.01), # GAP_EXTEND / COMPARE_FACTOR
]

In [10]:
discretization = 10
n_dims = len(x_bounds)
xs = [np.linspace(start, stop, discretization) for (start, stop) in x_bounds]
grids = np.meshgrid(*xs, indexing='ij')
values = np.zeros_like(grids[0])

for i, k in ProgressBar(max_value=len(values.flat))(
        enumerate(itertools.product(range(discretization), repeat=n_dims))):
    values[k] = objective([grids[j][k] for j in range(n_dims)],
                          gold_sentences, gold_alignments)

min_value = np.min(values)
min_locations = np.where(values == min_value)
print('Min optimisation objective value {}, found in {} points'
      .format(min_value, len(min_locations[0])))

100% (1000 of 1000) |#####################| Elapsed Time: 0:26:50 Time: 0:26:50


Min optimisation objective value 255.0, found in 3 points


In [11]:
for k in zip(*min_locations):
    x = [grids[0][k], grids[1][k], grids[2][k]]
    print(x)
    print(x2parameters(x))
    print()

[-1.0, -0.12, -0.22999999999999998]
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -1.0, 'GAP_OPEN': -0.34999999999999998, 'GAP_EXTEND': -0.22999999999999998, 'EXCHANGE': None}>

[-0.78000000000000003, -0.12, -0.12]
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -0.78000000000000003, 'GAP_OPEN': -0.23999999999999999, 'GAP_EXTEND': -0.12, 'EXCHANGE': None}>

[-0.56000000000000005, -0.12, -0.01]
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -0.56000000000000005, 'GAP_OPEN': -0.13, 'GAP_EXTEND': -0.01, 'EXCHANGE': None}>



Attempting to fine-tune the minima found

In [12]:
for k in zip(*min_locations):
    print('Starting at x = ', x)
    result = optimize.minimize(
        objective, [grids[0][k], grids[1][k], grids[2][k]],
        method='SLSQP',
        bounds=x_bounds,
        args=(gold_sentences, gold_alignments),
        options={'disp': True, 'maxiter': 500},
        callback=print)
    print(result)
    print(x2parameters(result.x))
    print()
    print()

Starting at x =  [-0.56000000000000005, -0.12, -0.01]
[-1.   -0.12 -0.23]
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 255.0
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1
     fun: 255.0
     jac: array([ 0.,  0.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 5
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([-1.  , -0.12, -0.23])
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -1.0, 'GAP_OPEN': -0.34999999999999998, 'GAP_EXTEND': -0.22999999999999998, 'EXCHANGE': None}>


Starting at x =  [-0.56000000000000005, -0.12, -0.01]
[-0.78 -0.12 -0.12]
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 255.0
            Iterations: 1
            Function evaluations: 5
            Gradient evaluations: 1
     fun: 255.0
     jac: array([ 0.,  0.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 5
 

Does not seem to work much.

#### 2.2.2 On more restricted boundaries, with discretization=10

In [13]:
x_bounds = [
    (-1, -.5), # COMPARE_ORIGIN / COMPARE_FACTOR
    (-.5, -.01), # (GAP_OPEN - GAP_EXTEND) / COMPARE_FACTOR
    (-.5, -.01), # GAP_EXTEND / COMPARE_FACTOR
]

In [14]:
discretization = 10
n_dims = len(x_bounds)
xs = [np.linspace(start, stop, discretization) for (start, stop) in x_bounds]
grids = np.meshgrid(*xs, indexing='ij')
values = np.zeros_like(grids[0])

for i, k in ProgressBar(max_value=len(values.flat))(
        enumerate(itertools.product(range(discretization), repeat=n_dims))):
    values[k] = objective([grids[j][k] for j in range(n_dims)],
                          gold_sentences, gold_alignments)

min_value = np.min(values)
min_locations = np.where(values == min_value)
print('Min optimisation objective value {}, found in {} points'
      .format(min_value, len(min_locations[0])))

100% (1000 of 1000) |#####################| Elapsed Time: 0:18:56 Time: 0:18:56


Min optimisation objective value 240.0, found in 1 points


In [15]:
for k in zip(*min_locations):
    x = [grids[0][k], grids[1][k], grids[2][k]]
    print(x)
    print(x2parameters(x))
    print()

[-0.88888888888888884, -0.17333333333333334, -0.11888888888888893]
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -0.88888888888888884, 'GAP_OPEN': -0.29222222222222227, 'GAP_EXTEND': -0.11888888888888893, 'EXCHANGE': None}>



#### 2.2.2 On more restricted boundaries, with discretization=20

In [16]:
x_bounds = [
    (-1, -.5), # COMPARE_ORIGIN / COMPARE_FACTOR
    (-.5, -.01), # (GAP_OPEN - GAP_EXTEND) / COMPARE_FACTOR
    (-.5, -.01), # GAP_EXTEND / COMPARE_FACTOR
]

In [17]:
discretization = 20
n_dims = len(x_bounds)
xs = [np.linspace(start, stop, discretization) for (start, stop) in x_bounds]
grids = np.meshgrid(*xs, indexing='ij')
values = np.zeros_like(grids[0])

for i, k in ProgressBar(max_value=len(values.flat))(
        enumerate(itertools.product(range(discretization), repeat=n_dims))):
    values[k] = objective([grids[j][k] for j in range(n_dims)],
                          gold_sentences, gold_alignments)

min_value = np.min(values)
min_locations = np.where(values == min_value)
print('Min optimisation objective value {}, found in {} points'
      .format(min_value, len(min_locations[0])))

100% (8000 of 8000) |#####################| Elapsed Time: 2:33:16 Time: 2:33:16


Min optimisation objective value 242.0, found in 2 points


In [18]:
for k in zip(*min_locations):
    x = [grids[0][k], grids[1][k], grids[2][k]]
    print(x)
    print(x2parameters(x))
    print()

[-0.97368421052631582, -0.16473684210526318, -0.16473684210526318]
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -0.97368421052631582, 'GAP_OPEN': -0.32947368421052636, 'GAP_EXTEND': -0.16473684210526318, 'EXCHANGE': None}>

[-0.92105263157894735, -0.16473684210526318, -0.13894736842105265]
<frozendict {'COMPARE_FACTOR': 1, 'COMPARE_ORIGIN': -0.92105263157894735, 'GAP_OPEN': -0.30368421052631583, 'GAP_EXTEND': -0.13894736842105265, 'EXCHANGE': None}>



TODO: why less with finer discretisation?